In [354]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as pyplot
import re
%matplotlib inline

In [355]:
salary_df = pd.read_csv('hawks_salary.csv')
salary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 13 columns):
No.           23 non-null object
Player        24 non-null object
Flag          24 non-null object
Pos           24 non-null object
Age           24 non-null int64
Ht            24 non-null object
Wt            24 non-null int64
S/C           24 non-null object
Exp           24 non-null object
Birth Date    24 non-null object
Summary       22 non-null object
Salary        23 non-null object
Draft         20 non-null object
dtypes: int64(2), object(11)
memory usage: 2.6+ KB


In [356]:
salary_df.head()

,No.,Player,Flag,Pos,Age,Ht,Wt,S/C,Exp,Birth Date,Summary,Salary,Draft
0,NaN,Nicolas Beaudin\beaudni01,ca,D,20,5-11,168,L/-,R,October 7 1999,NaN,NaN,2018 CHI 1st (27)
1,27,Adam Boqvist\boqviad01,se,D,19,5-11,179,R/-,R,August 15 2000,4 G 9 A 13 P,"$925,000",2018 CHI 1st (8)
2,91,Drake Caggiula\caggidr01,ca,F,25,5-10,176,L/-,3,June 20 1994,9 G 6 A 15 P,"$1,500,000",NaN
3,46-47,Lucas Carlsson\carlslu01,se,D,22,6-0,189,L/-,R,July 5 1997,0 G 1 A 1 P,"$792,500",2016 CHI 4th (110)
4,22,Ryan Carpenter\carpery01,us,F,29,6-0,200,R/-,4,January 18 1991,3 G 12 A 15 P,"$1,000,000",NaN


In [357]:
salary_df = salary_df[1:]

In [358]:
col_titles = ['no.', 'name', 'nationality', 'pos', 'age', 'ht', 'wt', 'hand', 'exp', 'birth_date', 'summary', 'salary', 'draft']
salary_df.columns = col_titles

In [359]:
names = salary_df.name.tolist()
new_names = []
for name in names:
    new_names.append(name.split("\\")[0])
    
salary_df['name'] = new_names

In [360]:
country_dict = {'se':'SWE', 'ca':'CAN', 'us':'USA', 'cz':'CZE', 'fi':'FIN','ru':'RUS', 'ge':'GER'}
salary_df.nationality = salary_df.nationality.replace(country_dict)

In [361]:
handedness = salary_df.hand.tolist()
new_hand = []
for x in handedness:
    new_hand.append(x.split("/")[0])

salary_df['hand'] = new_hand

In [362]:
salary = salary_df.salary.tolist()
new_salary = []
for x in salary:
    x = x.replace(",","")
    new_salary.append(x.split("$")[1])
    
salary_df['salary'] = new_salary
salary_df.salary = salary_df.salary.astype('int')

In [363]:
salary_df.exp.replace({'R':'0.5'}, inplace = True)

In [364]:
index_names = salary_df[(salary_df.name == 'Corey Crawford') | (salary_df.name == 'Malcolm Subban')].index
salary_df.drop(index_names, inplace = True)

In [365]:
heights = salary_df.ht.tolist()
new_heights = []
for x in heights:
    x = (int(x.split('-')[0]) * 12) + (int(x.split('-')[1]))
    new_heights.append(x)
    
salary_df.ht = new_heights

In [366]:
salary_df.drop(['summary', 'draft', 'birth_date'], axis = 1, inplace = True)

In [367]:
positions = ['F', 'D']
salary_df = salary_df[salary_df['pos'].isin(positions)]

In [382]:
salary_df.replace('Jonathan Toews (C)', 'Jonathan Toews', inplace = True)

In [383]:
salary_df

,no.,name,nationality,pos,age,ht,wt,hand,exp,salary,captain
1,27,Adam Boqvist,SWE,D,19,71,179,R,0.5,925000,0
2,91,Drake Caggiula,CAN,F,25,70,176,L,3,1500000,0
3,46-47,Lucas Carlsson,SWE,D,22,72,189,L,0.5,792500,0
4,22,Ryan Carpenter,USA,F,29,72,200,R,4,1000000,0
6,77,Kirby Dach,CAN,F,19,76,197,R,0.5,925000,0
7,12,Alex DeBrincat,USA,F,22,67,165,R,2,800000,0
8,38,Brandon Hagel,CAN,F,21,71,174,L,0.5,925000,0
9,36,Matthew Highmore,CAN,F,23,71,188,L,1,817500,0
10,64,David Kampf,CZE,F,25,74,188,L,2,1000000,0
11,88,Patrick Kane,USA,F,31,70,177,L,12,9800000,0


# Skaters DB

In [389]:
hawks_df = pd.read_csv('skaters.csv')
hawks_df = hawks_df[hawks_df.team == 'CHI']

In [390]:
hawks_df

,playerId,season,name,team,position,situation,games_played,icetime,shifts,gameScore,...,OffIce_F_xGoals,OffIce_A_xGoals,OffIce_F_shotAttempts,OffIce_A_shotAttempts,xGoalsForAfterShifts,xGoalsAgainstAfterShifts,corsiForAfterShifts,corsiAgainstAfterShifts,fenwickForAfterShifts,fenwickAgainstAfterShifts
150,8477846,2019,Ryan Carpenter,CHI,C,other,68,873,25,7.44,...,10.42,8.55,83,63,0.71,0.00,1,1,1,0
151,8477846,2019,Ryan Carpenter,CHI,C,all,68,57465,1320,12.50,...,159.20,160.96,3140,2999,0.00,0.00,0,0,0,0
152,8477846,2019,Ryan Carpenter,CHI,C,5on5,68,46118,996,12.50,...,104.51,119.10,2406,2521,0.85,1.02,35,31,21,26
153,8477846,2019,Ryan Carpenter,CHI,C,4on5,68,9604,268,10.92,...,3.12,13.18,33,240,0.21,0.00,2,1,2,0
154,8477846,2019,Ryan Carpenter,CHI,C,5on4,68,870,31,3.85,...,9.99,2.05,184,25,0.00,0.15,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4300,8481637,2019,Anton Wedin,CHI,L,other,4,0,0,0.00,...,0.00,0.00,0,0,0.00,0.00,0,0,0,0
4301,8481637,2019,Anton Wedin,CHI,L,all,4,2177,51,0.10,...,10.89,10.15,211,195,0.00,0.00,0,0,0,0
4302,8481637,2019,Anton Wedin,CHI,L,5on5,4,2177,51,0.10,...,5.88,7.37,159,154,0.20,0.01,5,1,4,1
4303,8481637,2019,Anton Wedin,CHI,L,4on5,4,0,0,0.00,...,0.00,0.00,0,0,0.00,0.00,0,0,0,0


In [391]:
hawks_names = hawks_df.name.unique()
salary_names = salary_df.name.unique()

not_in_salary = []
for name in hawks_names:
    if name not in salary_names:
        not_in_salary.append(name)
        
not_in_salary

['Brent Seabrook',
 'Dennis Gilbert',
 'Dylan Sikura',
 'Alex Nylander',
 'Calvin de Haan',
 'John Quenneville',
 'Andrew Shaw',
 'Anton Wedin']

In [392]:
not_in_hawks = []
for name in salary_names:
    if name not in hawks_names:
        not_in_hawks.append(name)

In [393]:
not_in_hawks

['Brandon Hagel', 'Alexander Nylander']